In [ ]:
# !pip install optuna

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

import optuna

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    MODEL_PATH = '/home/commonlit/models/deberta-large-lm/best_lm'
    TOKENIZER_PATH = '/home/commonlit/models/deberta-large-lm/best_lm'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#     DEVICE = "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'deberta-large-lm'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [12]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [13]:
with open('../data/tokenizer.vocab.txt', 'w') as f:
    for k, v in tokenizer.vocab.items():
        f.write(f'{k}: {v}\n')

In [14]:
pad_token = '______'

In [15]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
#         tokenizer.add_special_tokens({'pad_token': pad_token})
#         assert tokenizer.pad_token == pad_token
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        self.tokenizer = tokenizer
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [16]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [17]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [18]:
config = AutoConfig.from_pretrained(cfg.MODEL_PATH)

In [19]:
config.vocab_size, tokenizer.vocab_size

(50265, 50265)

In [20]:
from transformers import AutoModelForSequenceClassification

class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.MODEL_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(cfg.MODEL_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [21]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at /home/commonlit/models/deberta-large-lm/best_lm were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model

In [22]:
import re

for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('layer') > -1):
        layer_name = re.sub(r'.+(layer\.\d+).+', r'\1', name)

In [23]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.deberta.embeddings.word_embeddings.weight torch.Size([50265, 1024])
1 transformer_model.deberta.embeddings.LayerNorm.weight torch.Size([1024])
2 transformer_model.deberta.embeddings.LayerNorm.bias torch.Size([1024])
3 transformer_model.deberta.encoder.layer.0.attention.self.q_bias torch.Size([1024])
4 transformer_model.deberta.encoder.layer.0.attention.self.v_bias torch.Size([1024])
5 transformer_model.deberta.encoder.layer.0.attention.self.in_proj.weight torch.Size([3072, 1024])
6 transformer_model.deberta.encoder.layer.0.attention.self.pos_proj.weight torch.Size([1024, 1024])
7 transformer_model.deberta.encoder.layer.0.attention.self.pos_q_proj.weight torch.Size([1024, 1024])
8 transformer_model.deberta.encoder.layer.0.attention.self.pos_q_proj.bias torch.Size([1024])
9 transformer_model.deberta.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
10 transformer_model.deberta.encoder.layer.0.attention.output.dense.bias torch.Size([1024])
11 trans

In [24]:
# sample_input_ids = torch.randint(0, 1000, [2, 248])
# sample_attention_mask = torch.randint(0, 1000, [2, 248])

In [25]:
sample_records = [sample_ds[i] for i in range(2)]

In [26]:
sample_records[0].keys()

dict_keys(['input_ids', 'attention_mask', 'target'])

In [27]:
sample_input_ids = torch.stack([r['input_ids'] for r in sample_records])
sample_attention_mask = torch.stack([r['attention_mask'] for r in sample_records])

In [28]:
sample_input_ids.shape, sample_attention_mask.shape

(torch.Size([2, 248]), torch.Size([2, 248]))

In [29]:
sample_input_ids

tensor([[    1,  1779,     5,   664,    82,  1835,     7,     5,  1011,  4294,
             6,    24,  2633,    10, 27265,  1714,  2772,     4,  2978,     9,
            41,  6291,  1310,     6,    24,    21,    10,  2608,  5252,     4,
         50118,   133,  1929,    21,  2913,    19,  1958,    12,  9830, 20790,
             6,    45,  4976,    15, 17359,     6,    53, 11122, 18331,    81,
         24271,     8,  9910,  6368,     6,   101,    10,   588,  1958,   882,
             4,    20,  3617, 38325,     8,   655,   571, 18656,    14,    56,
         14633,     5,   929,     6,    58, 39143,    19, 15039,     8, 22246,
         11538,    19, 13145, 21811,     9, 13178,     6,   101,  1958,     4,
          1578, 11720,  8402,    56,    57, 14998, 38073,    15,   106,     6,
             8, 19053,   154, 16155, 41591, 20846, 10601,    31,     5,  9836,
             4, 50118,  3750,   349,   253,     9,     5,   929,     6,    15,
             5,  2204,     6, 10601,    10,  2721,  

In [30]:
internal_out = sample_model.transformer_model(sample_input_ids, attention_mask=sample_attention_mask)

In [31]:
internal_out.keys()

odict_keys(['logits', 'hidden_states'])

In [32]:
len(internal_out.hidden_states), internal_out.hidden_states[-1].shape

(25, torch.Size([2, 248, 1024]))

In [33]:
sample_res = sample_model(sample_input_ids, sample_attention_mask)

In [34]:
sample_res[0].shape, sample_res[1].shape

(torch.Size([2, 1]), torch.Size([2, 1024]))

In [35]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[-20.3559, -29.9483, -10.9850,  ...,   1.9756,  17.1997, -22.4863],
        [ -7.3677,  26.9104,  -5.2424,  ...,  39.6084,   6.2713,  -9.2617],
        [  1.1094, -15.0250,  -9.2055,  ...,  26.7118,   0.3870,   8.1409],
        ...,
        [ 23.0908,  35.5163,  -8.3487,  ...,   5.5515, -10.6900,  12.9882],
        [ -9.8833,  14.3958, -32.4952,  ..., -28.4682, -17.0043, -19.5385],
        [ -1.2294,  35.6013,  -9.9441,  ...,  36.4845,  19.7578,  25.7991]])

### Evaluation and Prediction

In [36]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [37]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [38]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [39]:
def create_optimizer(model, base_lr=5e-5, last_lr=None):
    named_parameters = list(model.named_parameters())
    attention_param_start = 388
    regressor_param_start = 392
    roberta_parameters = named_parameters[:attention_param_start]
    attention_parameters = named_parameters[attention_param_start:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    if last_lr is not None:
        parameters.append({"params": attention_group, "lr": last_lr})
        parameters.append({"params": regressor_group, "lr": last_lr})
    else:
        parameters.append({"params": attention_group})
        parameters.append({"params": regressor_group})
        
    # Change on different models
    layer_low_threshold = 195
    layer_middle_threshold = 323
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5 # 2e-05
        if layer_num >= layer_middle_threshold:
            lr = base_lr / 0.5 # 1e-4
        elif layer_num >= layer_low_threshold:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [40]:
sample_optimizer = create_optimizer(sample_model)

In [41]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [42]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [43]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
#         torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [44]:
class Trainer():
    def __init__(self, scaler, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.scaler, self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            scaler, model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        val_rmse_list = []
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                # Casts operations to mixed precision
                with torch.cuda.amp.autocast():
                    pred, _ = self.model(input_ids, attention_mask)
                    mse = mse_loss(pred.flatten(), target)
                    
                self.scaler.scale(mse).backward()
                self.scaler.step(self.optimizer)
                self.scaler.update()
                
#                 mse.backward()
#                 self.optimizer.step()
                
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, self.model, self.model_path)
                    val_rmse_list.append(val_rmse)
                    start = time.time()
                # Finish early on condition
                if epoch > 0 and best_val_rmse > 0.6 or (len(val_rmse_list) > 5 and np.array(val_rmse_list).mean() > 1.0):
                    return best_val_rmse
                
                step += 1
        return best_val_rmse

In [45]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = list(kfold.split(train_df))

### Optuna

In [46]:
del sample_model
gc.collect()
torch.cuda.empty_cache()

In [47]:
# Best results
# Fold 0: { 'base_lr': 0.0001190683694379101, 'last_lr': 0.00017987585986205585, 'epochs': 4 } Best value: 0.49271923303604126
# Fold 1: {'base_lr': 0.00012114635348406963, 'last_lr': 0.0005477206613438486, 'epochs': 4}. Best value:  0.45853328704833984
# Fold 2: {'base_lr': 5.24730490640746e-05, 'last_lr': 0.00020041362261812433, 'epochs': 4}   Best value:  0.49088865518569946
# Fold 3: {'base_lr': 6.108276630664184e-05, 'last_lr': 0.00011544056953737668, 'epochs': 4}. Best value:  0.4930591881275177
# Fold 4: {'base_lr': 0.0001717178883932075, 'last_lr': 0.00042448836147656634, 'epochs': 4}  Best value:  0.48955243825912476
# Fold 5: {'base_lr': 0.000135700916847811, 'last_lr': 0.0029640935672153, 'epochs': 4}.      Best value:  0.4688156247138977

In [48]:

fold = 0

def objective(trial):
    base_lr = trial.suggest_float("base_lr", 3e-5, 5e-4, log=True)
    last_lr = trial.suggest_float("last_lr", 8e-5, 5e-3, log=True)
    epochs = 4
    schedule_func = trial.suggest_categorical('schedule_func', [get_cosine_with_hard_restarts_schedule_with_warmup, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup])
    
    print(f'##### Using fold {fold}')
    print(f'##### Using base_lr {base_lr} last_lr {last_lr} epochs {epochs}')
    print(f'##### Using {schedule_func}')
    
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_indices, val_indices = splits[fold]
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    model = CommonLitModel().to(cfg.DEVICE)
    
    optimizer = create_optimizer(model, base_lr=base_lr, last_lr=last_lr)
    
    scheduler = schedule_func(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    scaler = torch.cuda.amp.GradScaler() # fp16
    
    trainer = Trainer(scaler, model, model_path, train_loader, val_loader, optimizer, 
                      scheduler = scheduler, num_epochs = epochs)
    rmse_val = trainer.train()
    
    del trainer
    del model
    del tokenizer
    del scaler
    del optimizer
    del train_loader
    del val_loader
    torch.cuda.empty_cache()
    gc.collect()
    
    return rmse_val

In [49]:
for i in range(0, 1):
    fold = i
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    print(" Best value: ", study.best_trial.value)
    print(" Best params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")

[I 2021-07-31 08:29:25,821] A new study created in memory with name: no-name-5ff454cc-d5de-4c84-9ab3-46c6ac4e8fe1


##### Using fold 0
##### Using base_lr 0.00036441540609173276 last_lr 0.003177859991698281 epochs 4
##### Using <function get_linear_schedule_with_warmup at 0x7f15aa1cc280>


Some weights of the model checkpoint at /home/commonlit/models/deberta-large-lm/best_lm were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model


16 steps took 9.01 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.7832 New best_val_rmse: 0.7832

16 steps took 6.63 seconds
Epoch: 0 batch_num: 32 val_rmse: 2.056 Still best_val_rmse: 0.7832 (from epoch 0)

16 steps took 6.62 seconds
Epoch: 0 batch_num: 48 val_rmse: 1.233 Still best_val_rmse: 0.7832 (from epoch 0)

16 steps took 6.58 seconds
Epoch: 0 batch_num: 64 val_rmse: 1.037 Still best_val_rmse: 0.7832 (from epoch 0)

16 steps took 6.55 seconds
Epoch: 0 batch_num: 80 val_rmse: 1.05 Still best_val_rmse: 0.7832 (from epoch 0)

16 steps took 6.56 seconds
Epoch: 0 batch_num: 96 val_rmse: 1.034 Still best_val_rmse: 0.7832 (from epoch 0)


[I 2021-07-31 08:30:48,130] Trial 0 finished with value: 0.783240556716919 and parameters: {'base_lr': 0.00036441540609173276, 'last_lr': 0.003177859991698281, 'schedule_func': <function get_linear_schedule_with_warmup at 0x7f15aa1cc280>}. Best is trial 0 with value: 0.783240556716919.


...


##### Using fold 0
##### Using base_lr 3.981758887999553e-05 last_lr 0.00040821877556246653 epochs 4
##### Using <function get_cosine_schedule_with_warmup at 0x7f15aa1cc310>


Some weights of the model checkpoint at /home/commonlit/models/deberta-large-lm/best_lm were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model


16 steps took 9.35 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9201 New best_val_rmse: 0.9201

16 steps took 6.75 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.6768 New best_val_rmse: 0.6768

16 steps took 6.82 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7356 Still best_val_rmse: 0.6768 (from epoch 0)

16 steps took 6.8 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6062 New best_val_rmse: 0.6062

16 steps took 6.82 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6594 Still best_val_rmse: 0.6062 (from epoch 0)

16 steps took 6.82 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.7561 Still best_val_rmse: 0.6062 (from epoch 0)

16 steps took 6.83 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6489 Still best_val_rmse: 0.6062 (from epoch 0)

16 steps took 6.83 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5917 New best_val_rmse: 0.5917

16 steps took 6.83 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.509 New best_val_rmse: 0.509

16 steps took 7.71 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5209 Still best_val_rms

[I 2021-07-31 11:57:50,560] Trial 19 finished with value: 0.4797709584236145 and parameters: {'base_lr': 3.981758887999553e-05, 'last_lr': 0.00040821877556246653, 'schedule_func': <function get_cosine_schedule_with_warmup at 0x7f15aa1cc310>}. Best is trial 17 with value: 0.4751589596271515.



 Best value:  0.4751589596271515
 Best params: 
    base_lr: 3.9152303052330085e-05
    last_lr: 0.00045881520547034934
    schedule_func: <function get_cosine_schedule_with_warmup at 0x7f15aa1cc310>


In [ ]:
%%time

for i in range(4, 5):
    fold = i
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    print(" Best value: ", study.best_trial.value)
    print(" Best params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")

[I 2021-07-31 11:57:50,573] A new study created in memory with name: no-name-73d48b9f-fa79-4594-bfb3-1bc81cf3326d


##### Using fold 4
##### Using base_lr 5.367902443436466e-05 last_lr 0.0017631891864399083 epochs 4
##### Using <function get_linear_schedule_with_warmup at 0x7f15aa1cc280>


Some weights of the model checkpoint at /home/commonlit/models/deberta-large-lm/best_lm were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model


16 steps took 9.31 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9587 New best_val_rmse: 0.9587

16 steps took 6.82 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8325 New best_val_rmse: 0.8325

16 steps took 6.81 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6585 New best_val_rmse: 0.6585

16 steps took 6.82 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7102 Still best_val_rmse: 0.6585 (from epoch 0)

16 steps took 6.83 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6447 New best_val_rmse: 0.6447

16 steps took 6.84 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5472 New best_val_rmse: 0.5472

16 steps took 6.86 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5518 Still best_val_rmse: 0.5472 (from epoch 0)

16 steps took 6.85 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5713 Still best_val_rmse: 0.5472 (from epoch 0)

16 steps took 6.84 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5624 Still best_val_rmse: 0.5472 (from epoch 0)

16 steps took 7.74 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5222 New best_val_rm

[I 2021-07-31 12:10:10,085] Trial 0 finished with value: 0.4794856011867523 and parameters: {'base_lr': 5.367902443436466e-05, 'last_lr': 0.0017631891864399083, 'schedule_func': <function get_linear_schedule_with_warmup at 0x7f15aa1cc280>}. Best is trial 0 with value: 0.4794856011867523.


...


16 steps took 8.44 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.178 New best_val_rmse: 1.178

16 steps took 6.79 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.099 New best_val_rmse: 1.099

16 steps took 6.67 seconds
Epoch: 0 batch_num: 48 val_rmse: 1.087 New best_val_rmse: 1.087

16 steps took 6.64 seconds
Epoch: 0 batch_num: 64 val_rmse: 1.039 New best_val_rmse: 1.039

16 steps took 6.64 seconds
Epoch: 0 batch_num: 80 val_rmse: 1.039 Still best_val_rmse: 1.039 (from epoch 0)

16 steps took 6.65 seconds
Epoch: 0 batch_num: 96 val_rmse: 1.035 New best_val_rmse: 1.035


[I 2021-07-31 13:57:48,023] Trial 10 finished with value: 1.0347208976745605 and parameters: {'base_lr': 0.00045174510734207005, 'last_lr': 0.004849030517443048, 'schedule_func': <function get_cosine_schedule_with_warmup at 0x7f15aa1cc310>}. Best is trial 5 with value: 0.4748414158821106.



##### Using fold 4
##### Using base_lr 3.758085868459081e-05 last_lr 0.0009708867541466488 epochs 4
##### Using <function get_cosine_schedule_with_warmup at 0x7f15aa1cc310>


Some weights of the model checkpoint at /home/commonlit/models/deberta-large-lm/best_lm were not used when initializing DebertaForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model


16 steps took 8.27 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8271 New best_val_rmse: 0.8271

16 steps took 6.79 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8755 Still best_val_rmse: 0.8271 (from epoch 0)

16 steps took 6.79 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7494 New best_val_rmse: 0.7494

16 steps took 6.79 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.8066 Still best_val_rmse: 0.7494 (from epoch 0)

16 steps took 6.79 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6462 New best_val_rmse: 0.6462

16 steps took 6.79 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5594 New best_val_rmse: 0.5594

16 steps took 6.79 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5968 Still best_val_rmse: 0.5594 (from epoch 0)

16 steps took 6.8 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.521 New best_val_rmse: 0.521

16 steps took 6.8 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5169 New best_val_rmse: 0.5169

16 steps took 7.73 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5397 Still best_val_rmse: 0.5169 (from ep

In [ ]:
for i in range(4, len(list(splits))):
    fold = i
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20)
    print(" Best value: ", study.best_trial.value)
    print(" Best params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")

### Verify the model

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [ ]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [ ]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [ ]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

In [ ]:
from transformers import RobertaTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = RobertaTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [ ]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [ ]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [ ]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [ ]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [ ]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [ ]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

In [ ]:
final_kernel_predictions_means

In [ ]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [ ]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

In [ ]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [ ]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

In [ ]:
final_scores_flat

In [ ]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

In [ ]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

### Prepare Packaging

In [ ]:
cfg.model_name

In [ ]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [ ]:
BEST_MODEL_FOLDER

In [ ]:
cfg.NUM_FOLDS

In [ ]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [ ]:
bestmodels

In [ ]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/vocab.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        merges = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/merges.txt'))
        assert merges.exists()
        copyfile(merges, tokenizer_path/'merges.txt')
    else:
        print(f'{best_model_file} is missing')

In [ ]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

In [ ]:
!ls {MODELS_PATH/cfg.model_name}

In [ ]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

In [ ]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [ ]:
!du -h {MODELS_PATH/cfg.model_name}/*

In [ ]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

In [ ]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

In [ ]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [ ]:
!cat {str(dataset_json_path)}

In [ ]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}-light').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}-light')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

In [ ]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [ ]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with merges.txt" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)